In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

WEEK_TAG = "credit_data_2026_W01"
RAW_DIR = Path("../data/raw") / WEEK_TAG

customers = pd.read_csv(RAW_DIR / "customers.csv", parse_dates=["created_date"])
accounts = pd.read_csv(RAW_DIR / "accounts.csv", parse_dates=["opened_date"])
snap = pd.read_csv(RAW_DIR / "weekly_account_snapshot.csv", parse_dates=["snapshot_date"])
tx = pd.read_csv(RAW_DIR / "transactions.csv", parse_dates=["transaction_date"])

print("customers:", customers.shape)
print("accounts:", accounts.shape)
print("snapshot:", snap.shape)
print("transactions:", tx.shape)


customers: (200000, 6)
accounts: (426278, 6)
snapshot: (426278, 10)
transactions: (233323, 5)


In [3]:
df = (
    snap.merge(accounts, on="account_id", how="left")
        .merge(customers, on="customer_id", how="left")
)

df.head()


,snapshot_date,account_id,balance,utilisation,min_due,payment_made,payment_amount,days_past_due,missed_payment_flag,default_flag,customer_id,product_type,credit_limit,apr,opened_date,age,region,employment_status,income_band,created_date
0,2026-01-01,1,1223.68,0.3418,44.28,1,94.65,0,0,0,1,credit_card,3580,0.233153,2021-11-17,23,NI,employed,20-35k,2021-06-08
1,2026-01-01,2,2387.83,0.4849,138.39,1,301.31,0,0,0,2,credit_card,4924,0.155281,2023-08-25,62,NI,employed,50-75k,2021-09-15
2,2026-01-01,3,1683.04,0.5977,56.23,0,0.00,0,1,0,3,credit_card,2816,0.290071,2023-04-09,55,NI,student,20-35k,2019-03-26
3,2026-01-01,4,886.29,0.2153,25.24,0,0.00,0,1,0,3,credit_card,4117,0.269008,2023-05-05,55,NI,student,20-35k,2019-03-26
4,2026-01-01,5,5074.67,0.6154,219.28,1,460.70,0,0,0,4,personal_loan,8246,0.192752,2022-09-29,43,NI,employed,35-50k,2020-10-16


In [4]:
def quick_health(df, key_cols):
    out = {}
    out["rows"] = len(df)
    for c in key_cols:
        out[f"{c}_null_rate"] = df[c].isna().mean()
        out[f"{c}_nunique"] = df[c].nunique()
    return pd.Series(out)

health = quick_health(df, ["account_id", "customer_id", "utilisation", "balance", "days_past_due", "default_flag"])
health


rows                       426278.0
account_id_null_rate            0.0
account_id_nunique         426278.0
customer_id_null_rate           0.0
customer_id_nunique        200000.0
utilisation_null_rate           0.0
utilisation_nunique          9141.0
balance_null_rate               0.0
balance_nunique            258160.0
days_past_due_null_rate         0.0
days_past_due_nunique           5.0
default_flag_null_rate          0.0
default_flag_nunique            2.0
dtype: float64

In [5]:
df[["utilisation","balance","min_due","payment_amount","days_past_due"]].describe().T


,count,mean,std,min,25%,50%,75%,max
utilisation,426278.0,0.374788,0.165539,0.0068,0.2486,0.3572,0.4839,1.00
balance,426278.0,1821.757076,1437.033732,10.5400,830.0550,1438.3800,2374.2475,16729.37
min_due,426278.0,73.358136,62.956401,15.0000,30.2700,54.8000,94.9000,810.60
payment_amount,426278.0,98.721291,114.529060,0.0000,23.9700,65.7100,135.2500,1759.37
days_past_due,426278.0,2.542167,8.289006,0.0000,0.0000,0.0000,0.0000,60.00


In [6]:
kpis = pd.DataFrame({
    "snapshot_date": [df["snapshot_date"].min().date()],
    "accounts": [df["account_id"].nunique()],
    "customers": [df["customer_id"].nunique()],
    "default_rate": [df["default_flag"].mean()],
    "missed_payment_rate": [df["missed_payment_flag"].mean()],
    "dpd_30_plus_rate": [(df["days_past_due"] >= 30).mean()],
    "dpd_90_plus_rate": [(df["days_past_due"] >= 90).mean()],
    "avg_utilisation": [df["utilisation"].mean()],
    "median_utilisation": [df["utilisation"].median()],
    "avg_balance": [df["balance"].mean()],
})

kpis


,snapshot_date,accounts,customers,default_rate,missed_payment_rate,dpd_30_plus_rate,dpd_90_plus_rate,avg_utilisation,median_utilisation,avg_balance
0,2026-01-01,426278,200000,0.009818,0.201561,0.039695,0.0,0.374788,0.3572,1821.757076


In [7]:
df["risk_band"] = pd.cut(
    df["days_past_due"],
    bins=[-1, 0, 29, 89, 180],
    labels=["Current (0)", "Early (1-29)", "Late (30-89)", "Severe (90+)"]
)

band_summary = (
    df.groupby("risk_band")
      .agg(
          accounts=("account_id","count"),
          default_rate=("default_flag","mean"),
          missed_payment_rate=("missed_payment_flag","mean"),
          avg_util=("utilisation","mean"),
          avg_balance=("balance","mean")
      )
      .reset_index()
)

band_summary


/var/folders/rt/0zxshr9s4g713_r6y5sjpqk80000gn/T/ipykernel_13280/3624724345.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("risk_band")


,risk_band,accounts,default_rate,missed_payment_rate,avg_util,avg_balance
0,Current (0),362227,0.009870,0.201236,0.374761,1820.970377
1,Early (1-29),47130,0.009293,0.202504,0.374653,1824.332477
2,Late (30-89),16921,0.010165,0.205898,0.375745,1831.424640
3,Severe (90+),0,NaN,NaN,NaN,NaN


In [8]:
df["util_band"] = pd.cut(
    df["utilisation"],
    bins=[-0.001, 0.1, 0.3, 0.5, 0.7, 0.9, 1.0],
    labels=["0-10%","10-30%","30-50%","50-70%","70-90%","90-100%"]
)

util_summary = (
    df.groupby("util_band")
      .agg(
          accounts=("account_id","count"),
          default_rate=("default_flag","mean"),
          missed_payment_rate=("missed_payment_flag","mean"),
          avg_dpd=("days_past_due","mean")
      )
      .reset_index()
)

util_summary


/var/folders/rt/0zxshr9s4g713_r6y5sjpqk80000gn/T/ipykernel_13280/1937889035.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("util_band")


,util_band,accounts,default_rate,missed_payment_rate,avg_dpd
0,0-10%,8379,0.006803,0.127820,2.543263
1,10-30%,148624,0.009433,0.187729,2.537073
2,30-50%,173570,0.009927,0.205525,2.540848
3,50-70%,79682,0.010115,0.219937,2.544113
4,70-90%,15383,0.011766,0.232854,2.610999
5,90-100%,640,0.025000,0.264062,2.171875
